In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
from collections import defaultdict

In [3]:

dir = './runs'
directory = os.fsencode('./runs')

names = []
dfs = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    df = pd.read_csv(os.path.join(dir, filename))
    dfs.append(df)
    names.append(filename)


In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(14, 6)
ax.set(xlabel='Steps', ylabel='Eval F1', title='Eval F1 vs. steps')
ax.grid()

for idx, df in enumerate(dfs):
    label = names[idx].split('-0')[0][4:]
    plt.plot(df['Step'], df['Value'], label=label)
ax.legend()
axes = plt.gca()
plt.yticks(np.arange(64, 92, 2))
# fig.savefig("eval_f1_runs.png")

In [2]:
root_dir = './save/output'
directory = os.fsencode(root_dir)

d = defaultdict(defaultdict)

for dir in os.listdir(directory):
    model_dir = os.fsdecode(dir)
    with open(os.path.join(root_dir, model_dir, 'model_output.json')) as f:
        x = json.load(f)
        model_name = x['model_name'].replace('-dev', '').replace('-train', '').replace('-01', '')
        d[model_name][x['type']] = np.array(x['output'])
        del x
    



In [3]:
train = []
dev = []
for k, v in d.items():
    train.append(v['train'])
    dev.append(v['dev'])
    print(k)
    
    


albert-xxlarge-v1
albert-large-v2
roberta-large
bert-large


In [4]:
np_train = np.array(train).transpose(1, 0, 2, 3)
np_train.shape

(130319, 4, 2, 256)

In [5]:
np_dev = np.array(dev).transpose(1, 0, 2, 3)
np_dev.shape

(6078, 4, 2, 256)

In [8]:
import h5py
with h5py.File(os.path.join(root_dir, 'model_output.h5'), 'w') as hf:
    hf.create_dataset("model_output_train",  data=np_train)
    hf.create_dataset("model_output_dev",  data=np_dev)

In [10]:
with h5py.File(os.path.join(root_dir, 'model_output.h5'), 'r') as hf:
    data_train = hf['model_output_train'][:]
    data_dev = hf['model_output_dev'][:]
    print(data_train.shape, data_dev.shape)

(130319, 4, 2, 256) (6078, 4, 2, 256)
